In [ ]:
!pip install -U featuretools
!pip install nvcc4jupyter
#!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.2/619.2 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.4/754.4 kB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.1/236.1 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: holidays
    Found existing installation: holidays 0.41
    Uninstalling holidays-0.41:
      Successfully uninstalled holidays-0.41
Source files will be saved in "/tmp/tmpbyz7ujpy".


In [7]:
%%writefile main.cu
#include <stdio.h>
#include <iostream>
#include <stdexcept>
#include <cassert>
#include <stdexcept>
#include <iomanip>
//# include <cmath>


float round_to_one_decimal_place(float number) {
  return (static_cast<float>(static_cast<int>(number * 10.)) / 10.);
}

float round_to_three_decimal_place(float number) {
  return (static_cast<float>(static_cast<int>(number * 1000.)) / 1000.);
}

__device__ float* get_row(float* values, int row, int n) {

    float* row_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      int values_index = (row * n) + i;
      row_values[i] = values[values_index];
    }
    return row_values;
  }

  __device__ float* get_column(float* values, int column, int n) {

    float* column_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      int values_index = column + (i * n);
      column_values[i] = values[values_index];
    }
    return column_values;
  }

  __device__ float dot_product(float* v1, float* v2, int n) {

    float result = 0.0;
    for (int i = 0; i < n; i++) {
      result += (v1[i] * v2[i]);
    }

    return result;
  }

  __device__ void multiply_by_constant(float* values, int size, double constant) {

    for (int i = 0; i < size; i++) {
      values[i] = values[i] * constant;
    }

  }

  __device__ float* bracket_w(float x, float y, float z) {
    /* See page 113 of Modern Robotics.
    * HERE
    */

    int num_entries_in_matrix = 9;
    float* values = (float*) malloc(sizeof(float) * num_entries_in_matrix);

    // First row.
    values[0] = 0.;
    values[1] = -z;
    values[2] = y;

    // Second row.
    values[3] = z;
    values[4] = 0;
    values[5] = -x;

    // Third row.
    values[6] = -y;
    values[7] = x;
    values[8] = 0.;

    return values;
  }

  __device__ float* bracket_w_squared(float x, float y, float z) {
    /* See page 113 of Modern Robotics.
    * https://www.symbolab.com/solver/matrix-multiply-calculator
    */
    float* values = (float*) malloc(sizeof(float) * 9);

    // First row.
    values[0] = -(z * z) - (y * y);
    values[1] = y * x;
    values[2] = z * x;

    // Second row.
    values[3] = y * x;
    values[4] = -(z * z) - (x * x);
    values[5] = z * y;

    // Third row.
    values[6] = z * x;
    values[7] = z * y;
    values[8] = -(y * y) - (x * x);

    return values;
  }

  struct Twist {
  Twist(): wx(0.), wy(0.), wz(0.), vx(0.), vy(0.), vz(0.) {

  }

  Twist(float wx, float wy, float wz, float vx, float vy, float vz): wx(wx), wy(wy), wz(wz), vx(vx), vy(vy), vz(vz) {

  }

  private:
    float wx;
    float wy;
    float wz;
    float vx;
    float vy;
    float vz;
};

struct Screw {
  Screw(): wx(0.), wy(0.), wz(0.), vx(0.), vy(0.), vz(0.) {

  }

  Screw(float wx, float wy, float wz, float vx, float vy, float vz): wx(wx), wy(wy), wz(wz), vx(vx), vy(vy), vz(vz) {

  }

  __device__ Twist create_twist(float theta) {
    return Twist(wx * theta, wy * theta, wz * theta, vx * theta, vy * theta, vz * theta);
  }

  private:
    float wx;
    float wy;
    float wz;
    float vx;
    float vy;
    float vz;
};

  __device__ float* exponential_rotation(float x, float y, float z, float theta) {

    // I + sin θ[ωˆ] + (1 − cos θ)[ωˆ]2
    // A + B + C

    int num_entries_in_matrix = 9;
    float* B = bracket_w(x, y, z);
    float* C = bracket_w_squared(x, y, z);
    multiply_by_constant(B, num_entries_in_matrix, sin(theta));
    multiply_by_constant(C, num_entries_in_matrix, 1.0 - cos(theta));

    // Add in the component from the identity matrix.
    B[0] += 1.0;
    B[4] += 1.0;
    B[8] += 1.0;

    for (int i = 0; i < num_entries_in_matrix; i++) {
      B[i] += C[i];
    }

    // Cleanup memory here.
    free(C);

    return B;
  }

   __device__ float* rigid_body_exponential_translation(float wx, float wy, float wz, float vx, float vy, float vz, float theta) {

      // (Iθ + (1 − cos θ)[ω] + (θ − sin θ)[ω]2)v
      // (A + B + C) * (v)
      // (3 x 3 matrix) * (3 x 1 matrix) = (3 x 1 matrix)

      float* result = (float*) malloc(sizeof(float) * 3);
      int num_entries_in_matrix = 9;
      float* B = bracket_w(wx, wy, wz);
      float* C = bracket_w_squared(wx, wy, wz);
      multiply_by_constant(B, num_entries_in_matrix, 1.0 - std::cos(theta));
      multiply_by_constant(C, num_entries_in_matrix, theta - std::sin(theta));

      // Add the component due to A.
      B[0] += theta;
      B[4] += theta;
      B[8] += theta;

      // Add C.
      for (int i = 0; i < num_entries_in_matrix; i++) {
        B[i] += C[i];
      }

      // Multiply.
      result[0] = (B[0] * vx) + (B[1] * vy) + (B[2] * vz);
      result[1] = (B[3] * vx) + (B[4] * vy) + (B[5] * vz);
      result[2] = (B[6] * vx) + (B[7] * vy) + (B[8] * vz);

      return result;
   }

  __device__ float* rigid_body_exponential(float* screw, float theta) {
    return rigid_body_exponential(screw[0], screw[1], screw[2], screw[3], screw[4], screw[5], theta);
  }

  __device__ void normalize(float& x, float& y, float& z) {

    float magnitude = (x * x) + (y * y) + (z * z);
    x = x / magnitude;
    y = y / magnitude;
    z = z / magnitude;
  }

  __device__ void forward_kinematics(void) {


  }

__device__ float* rigid_body_exponential(float wx, float wy, float wz, float vx, float vy, float vz, float theta) {

    // See page 113 of Modern Robotics.

    int num_entries_in_matrix = 4 * 4;
    float* result = (float*) malloc(sizeof(float) * num_entries_in_matrix);

    normalize(wx, wy, wz);
    float* rotation_matrix = exponential_rotation(wx, wy, wz, theta);
    float* translation = rigid_body_exponential_translation(wx, wy, wz, vx, vy, vz, theta);

    // Write the rotation matrix to the result.
    result[0] = rotation_matrix[0];
    result[1] = rotation_matrix[1];
    result[2] = rotation_matrix[2];

    result[4] = rotation_matrix[3];
    result[5] = rotation_matrix[4];
    result[6] = rotation_matrix[5];

    result[8] = rotation_matrix[6];
    result[9] = rotation_matrix[7];
    result[10] = rotation_matrix[8];

    // Write the translation component to the result.
    result[3] = translation[0];
    result[7] = translation[1];
    result[11] = translation[2];

    // Write the last row of the matrix.
    result[12] = 0.0;
    result[13] = 0.0;
    result[14] = 0.0;
    result[15] = 1.0;

    // Free memory here.

    return result;
  }

__global__ void rigid_body_exponential_rotation_kernel(Screw** screw_axes, float** thetas, int num_joints, float** outputs) {

  // Allocate space for the twists.
  float** twists = (float**) malloc(sizeof(float*) * num_joints);

  // Create the twists.
  for (int i = 0; i < num_joints; i++) {
    twists[i] = rigid_body_exponential(screw_axes[i], thetas[i]);
  }

  float* left = twists[0];
  for (int i = 1; i < num_joints; i++) {
    float* result = (float*) malloc(sizeof(float) * 16);
    multiply_on_device(transformation, twists[i], 16, result)
    left = result;
  }

  int num_entries_in_matrix = 4 * 4;
  for (int i = 0; i < num_entries_in_matrix; i++) {
    output[i] = result[i];
  }
}


struct Vector3 {
  Vector3(): x(0.0), y(0.0), z(0.0) {

  }

  Vector3(float x, float y, float z): x(x), y(y), z(z) {

  }

  private:
    float x;
    float y;
    float z;
};


__device__ void multiply_on_device(float* left, float* right, int n, float* result) {

  int row_index = (blockDim.x * blockIdx.x) + threadIdx.x;
  int column_index = (blockDim.y * blockIdx.y) + threadIdx.y;
  int original_column_index = column_index;
  int offset_x = gridDim.x * blockDim.x;
  int offset_y = gridDim.y * blockDim.y;

  while (row_index < n) {

    float* v1 = get_row(left, row_index, n);
    while (column_index < n) {

      int result_index = (n * row_index) + column_index;
      float* v2 = get_column(right, column_index, n);
      result[result_index] = dot_product(v1, v2, n);

      // Increment the row and column indices.
      column_index += offset_y;
    }
    row_index += offset_x;
    column_index = original_column_index;
  }
}


__global__ void multiply(float* left, float* right, int n, float* result) {

  multiply_on_device(left, right, n, result);
}

struct SquareMatrix {

  int n;
  float* host_values;
  float* device_values;
  //~Matrix() = delete;

  SquareMatrix(int n): n(n) {

    // Allocate space on the host.
    host_values = (float*) malloc(sizeof(float) * n * n);

    // Allocate space on the device.
    cudaMalloc((void**)&device_values, sizeof(float) * n * n);

    for (int i = 0; i < n; i++) {
      for (int j = 0; j < n; j++) {
        int index = (i * n) + j;
        if (i == j) {
          host_values[index] = 1.0;
        }
        else {
          host_values[index] = 0.0;
        }
      }
    }
  }

  SquareMatrix write(float value, int row, int column) {

    int linear_index = (row * n) + column;
    if (linear_index < 0 || linear_index > (n * n)) {
      throw std::invalid_argument("Illegal (row, column) pair.");
    }

    host_values[linear_index] = value;
  }

  __device__ SquareMatrix gpu_write(float value, int row, int column) {

    int linear_index = (row * n) + column;
    host_values[linear_index] = value;
  }

  void print() {

    for (int i = 0; i < n; i++) {
      for (int j = 0; j < n; j++) {
        int linear_index = (i * n) + j;
        float rounded_value = round_to_three_decimal_place(host_values[linear_index]);
        std::cout << rounded_value << " ";
      }
      std::cout << std::endl;
    }
  }

  __device__ void gpu_write_row(float* values, int row) {

    int start_linear_index = (row * n);
    for (int i = 0; i < n; i++) {
      int device_index = start_linear_index + i;
      device_values[device_index] = values[i];
    }
  }

  void write_row(float* values, int row) {

    int start_linear_index = (row * n);
    for (int i = 0; i < n; i++) {
      int host_index = start_linear_index + i;
      host_values[host_index] = values[i];
    }
  }

  float get_value(int row, int column) const {

      int linear_index = (row * n) + column;
      if (linear_index > (n * n) || linear_index < 0) {
        throw std::invalid_argument("Illegal (row, column) pair.");
      }
      return host_values[linear_index];
  }

  float* get_row(int row) const {

    float* row_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      row_values[i] = get_value(row, i);
    }
    return row_values;
  }

  float* get_column(int column) const {
    float* column_values = (float*) malloc(sizeof(float) * n);
    for (int i = 0; i < n; i++) {
      column_values[i] = get_value(i, column);
    }
    return column_values;
  }

  float dot_product(const SquareMatrix& right, int row, int column) const {

    float* row_values = get_row(row);
    float* column_values = right.get_column(column);

    float result = 0.0;
    for (int i = 0; i < n; i++) {
      result += (row_values[i] * column_values[i]);
    }

    free(row_values);
    free(column_values);
    return result;
  }

  SquareMatrix operator*(const SquareMatrix& right) {

    // Multiply the matrices on the CPU.
    SquareMatrix result = SquareMatrix(right.n);

    for (int row = 0; row < n; row++) {
      for (int column = 0; column < n; column++) {

        float value = dot_product(right, row, column);
        result.write(value, row, column);
      }
    }

    return result;
  }

  SquareMatrix gpu_multiply(const SquareMatrix& right) {
    // Multiply the matrices on the GPU.
    SquareMatrix result = SquareMatrix(right.n);

    // Copy the values onto the GPU.
    cudaMemcpy(device_values, host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(right.device_values, right.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(result.device_values, result.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);

    // Run the kernel.
    dim3 blocks(2, 2);
    dim3 threads(80, 8);
    multiply<<<blocks, threads>>>(device_values, right.device_values, n, result.device_values);

    // Copy the result back from the device.
    cudaMemcpy(result.host_values, result.device_values, sizeof(float) * n * n, cudaMemcpyDeviceToHost);

    return result;
  }
};

struct Transformation: public SquareMatrix {

  Transformation(): SquareMatrix(4) {
  }

  Transformation(float* values): SquareMatrix(4) {

    for (int i = 0; i < n * n; i++) {
      host_values[i] = values[i];
    }
  }

  Transformation operator*(const Transformation& right) {

    // Multiply the matrices on the CPU.
    Transformation result = Transformation();

    for (int row = 0; row < n; row++) {
      for (int column = 0; column < n; column++) {

        float value = dot_product(right, row, column);
        result.write(value, row, column);
      }
    }

    return result;
  }

  Transformation gpu_multiply(const Transformation& right) {
    // Multiply the matrices on the GPU.
    Transformation result = Transformation();

    // Copy the values onto the GPU.
    cudaMemcpy(device_values, host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(right.device_values, right.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);
    cudaMemcpy(result.device_values, result.host_values, sizeof(float) * n * n, cudaMemcpyHostToDevice);

    // Run the kernel.
    dim3 blocks(2, 2);
    dim3 threads(80, 8);
    multiply<<<blocks, threads>>>(device_values, right.device_values, n, result.device_values);

    // Copy the result back from the device.
    cudaMemcpy(result.host_values, result.device_values, sizeof(float) * n * n, cudaMemcpyDeviceToHost);

    return result;
  }
};

float* create_joint_angles(float a, float b, float c, float d, float e, float f) {
  float* angles = (float*) malloc(sizeof(float) * 6);
  angles[0] = a;
  angles[1] = b;
  angles[2] = c;
  angles[3] = d;
  angles[4] = e;
  angles[5] = f;
  return angles;
}

#define N 1000000
#define num_threads_per_block 1024

int main() {

  // I know the screw axis and the angle of rotation about it.
  // How do I recover the transformation matrix?
  // Then I multipy all the transformation matrices in order.
  // Matrix exponential.
  // See page 105 of Modern Robotics
  // https://hades.mech.northwestern.edu/images/7/7f/MR.pdf
  // Page 113 contains the exponetial equation of [w]* theta

  // Robot info.
  int num_joints = 6;
  int num_configurations = 1;

  // Create the output arrays.
  float outputs_size = 16 * num_configurations;
  float* output_cpu = (float*) malloc(sizeof(float) * outputs_size);
  float* output_device;
  cudaMalloc((void**)&output_device, sizeof(float) * outputs_size);
  for (int i = 0; i < output_size; i++) {
    output_cpu[i] = 0.;
  }

  // Copy the outputs onto the device.
  cudaMemcpy(outputs_device, outputs_cpu, sizeof(float) * 16 * num_configurations, cudaMemcpyHostToDevice);

  Screw s1 = Screw([0., 2., 1., 0., 1., -2.]);
  Screw s2 = Screw([0., 2., 1., 0., 1., -2.]);
  Screw s3 = Screw([0., 2., 1., 0., 1., -2.]);
  Screw s4 = Screw([0., 2., 1., 0., 1., -2.]);
  Screw s5 = Screw([0., 2., 1., 0., 1., -2.]);
  Screw s6 = Screw([0., 2., 1., 0., 1., -2.]);
  Screw** screw_axes = (Screw**) malloc(sizeof(Screw*) * num_joints);

  float** thetas = (float**) malloc(sizeof(float*) * num_configurations);
  float* config_one_joint_angles = create_joint_angles(1., 2., 3., 4., 5., 6.);

  screw_axes[0] = &s1;
  screw_axes[1] = &s2;
  screw_axes[2] = &s3;
  screw_axes[3] = &s4;
  screw_axes[4] = &s5;
  screw_axes[5] = &s6;

  thetas[0] = config_one_joint_angles;
  // Add more thetas here.


  // [ 0.          0.89442719  0.4472136   0.          0.4472136  -0.89442719]
  // rigid_body_exponential_rotation_kernel(Screw** screw_axes, float** thetas, int num_joints, float** outputs)
  rigid_body_exponential_rotation_kernel<<<1, 1>>>(screw_axes, thetas, num_joints, outputs);

  // Copy the output back to the host.
  cudaMemcpy(outputs_cpu, outputs_device, sizeof(float) * 6 * num_thetas, cudaMemcpyDeviceToHost);

  std::vector<Transformation> transformations = std::Vector<Transformation>();
  for (int i = 0; i < num_configurations; i++) {
    float* matrix = outputs_cpu + (16 * i);
    Transformation result = Transformation(matrix);
  }

  transformations[0].print();

  return 0;
}

Overwriting main.cu


In [5]:
%%script bash
nvcc main.cu -o forward_kinematics

main.cu(164): error: argument of type "float" is incompatible with parameter of type "float *"
      return rigid_body_exponential(screw[0], screw[1], screw[2], screw[3], screw[4], screw[5], theta);
                                    ^

main.cu(164): error: too many arguments in function call
      return rigid_body_exponential(screw[0], screw[1], screw[2], screw[3], screw[4], screw[5], theta);
                                                        ^

main.cu(233): error: identifier "transformation" is undefined
      multiply_on_device(transformation, twists[i], 16, result)
                         ^

main.cu(233): error: identifier "multiply_on_device" is undefined
      multiply_on_device(transformation, twists[i], 16, result)
      ^

main.cu(234): error: expected a ";"
      left = result;
      ^

main.cu(239): error: identifier "result" is undefined
      output[i] = result[i];
                  ^

main.cu(528): error: argument of type "float *" is incompatible with parameter 

CalledProcessError: Command 'b'nvcc main.cu -o forward_kinematics\n'' returned non-zero exit status 2.

In [26]:
%%script bash
./forward_kinematics

0 -0.447 0.894 -0.999 
0.447 0.8 0.399 0.447 
-0.894 0.399 0.2 -0.894 
0 0 0 1 
Works!
